Proof of <a class="ProveItLink" href="../../../../../../_theory_nbs_/theory.ipynb">proveit</a>.<a class="ProveItLink" href="../../../../../_theory_nbs_/theory.ipynb">physics</a>.<a class="ProveItLink" href="../../../../_theory_nbs_/theory.ipynb">quantum</a>.<a class="ProveItLink" href="../../theory.ipynb">QPE</a>.<a class="ProveItLink" href="../../theorems.ipynb#psi_prime_t_lit_formula">psi_prime_t_lit_formula</a> theorem
========

In [ ]:
import proveit
theory = proveit.Theory() # the theorem's theory
from proveit import a, alpha, b, c, d, f, g, j, k, m, n, r, t, x, y, z, P, defaults, Function
from proveit.core_expr_types import fk, gk
from proveit.linear_algebra import (distribute_tensor_prod_over_sum, factor_complex_scalar_from_tensor_prod,
                                    scalar_tensor_associativity, tensor_prod_linearity)
from proveit.logic import Equals, InSet
from proveit.numbers import zero, one, two, e, i, pi, Natural, NaturalPos
from proveit.numbers import Add, Exp, Interval, Less, LessEq, Mult, Neg, subtract, Sum
from proveit.numbers.exponentiation import exponential_monotonocity, add_one_right_in_exp #exp_eq_for_eq_base_and_exp, 
from proveit.numbers.number_sets.natural_numbers import fold_forall_natural_pos
from proveit.numbers.summation import distributive_summation_spec
from proveit.physics.quantum import (Ket, multi_tensor_prod_induct_0, multi_tensor_prod_induct_1,
                                     RegisterKet, scalar_id_for_ket )
from proveit.physics.quantum.QPE import (
    p_prime_r_def, phase_, phase_is_real, psi_prime, psi_prime_expansion, psi_prime_t_def, t_,
    t_in_natural_pos, two_pow_t_is_nat_pos, two_pow_t_less_one_is_nat_pos)
from proveit.physics.quantum.QPE.phase_est_ops import SubIndexed

In [ ]:
%proving psi_prime_t_lit_formula

The theorem utilizes the _literal_ $t$ (representing the number of qubits in the first register (see Nielsen & Chuang's Figure 5.2, pg 222). We first prove the formula by induction on a _variable_ $t \in \mathbb{N}^{+}$, then at the end instantiate that result with $t_{\text{var}}\mapsto t_{\text{lit}}$. The variable $t$ is simply $t$ in this notation; the _literal_ $t$ is obtained by using $t\_$.

In [ ]:
# the formula using a variable $t$
psi_prime_var_t_formula = (
    Equals(RegisterKet(psi_prime, t),
           Sum(k, Mult(Exp(e, Mult(two, pi, i, phase_, k)), RegisterKet(k, t)),
               domain=Interval(zero, subtract(Exp(two, t), one)))))

In [ ]:
# Notice for the induction, we are assuming that the variable t is NaturalPos
defaults.assumptions = [InSet(t, NaturalPos)]

In [ ]:
# the induction theorem for positive naturals
fold_forall_natural_pos

In [ ]:
# instantiate the induction theorem for the variable t formula
induction_inst = fold_forall_natural_pos.instantiate(
    {Function(P,t):psi_prime_var_t_formula, m:t, n:t})

### Some Related Properties and Definitions Needed for Later Processing
Mainly: some domains and orderings. Notice that throughout the notebook, $t$ is a _variable_, not a literal.

In [ ]:
# used when processing products involving the phase phi
phase_is_real

In [ ]:
# named for convenience
two_pow_t_less_one = subtract(Exp(two, t), one)

In [ ]:
# needed for the next cell in which we prove that 0 ≤ 2^t - 1
two_pow_t_less_one.deduce_in_number_set(Natural)

In [ ]:
# needed later for a Sum.split() method in the induction step
LessEq(zero, two_pow_t_less_one).prove()

In [ ]:
Less(t, Add(t, one)).prove()

In [ ]:
exponential_monotonocity

In [ ]:
exponential_monotonocity_inst = exponential_monotonocity.instantiate({a: two, b: t, c: Add(t, one)})

In [ ]:
# needed later for a Sum.split() method in the induction step,
# allowing a splitting of a summation into the sum of two summations
exponential_monotonocity_inst.derive_shifted(Neg(one))

For later summation index manipulations, we want to establish that $2^{t+1}-2^{t} = 2^{t}$ (and more specifically we will need $2^{t+1}-2^{t}-1 = 2^{t}-1$).

In [ ]:
add_one_right_in_exp

In [ ]:
two_to_quant_t_plus_1_factored = add_one_right_in_exp.instantiate(
        {a: two, b: t}).derive_reversed()

In [ ]:
index_shift_simplification = two_to_quant_t_plus_1_factored.substitution(
        subtract(Exp(two, Add(t, one)), Exp(two, t)))

In [ ]:
index_shift_simplification = index_shift_simplification.inner_expr().rhs.factor(
        Exp(two, t)).simplify()

In [ ]:
# index_shift_simplification = index_shift_simplification.inner_expr().rhs.simplify()

In [ ]:
index_shift_simplification = index_shift_simplification.right_add_both_sides(Neg(one))

In [ ]:
index_shift_simplification = index_shift_simplification.inner_expr().lhs.commute(init_idx=1, final_idx=2)

In [ ]:
index_shift_simplification = index_shift_simplification.inner_expr().lhs.associate(start_idx=0, length=2)

In [ ]:
# Then clean up the resulting rhs
index_shift_simplification = index_shift_simplification.inner_expr().rhs.with_subtraction_at(1)

### Base Case

In [ ]:
base_case = induction_inst.antecedent.operands[0]

Axiomatically, $\psi'_{t}$ is defined as a tensor product:

In [ ]:
psi_prime_t_def

For $\psi'_{1}$, we prove a useful equality then instantiate the `psi_prime_t_def` with $t=1$:

In [ ]:
# this helps later with an instantiation and simplification,
# pre-establishing that 0 = -(1-1)
alt_reduction = Equals(zero, Neg(subtract(one, one))).prove()

In [ ]:
psi_prime_1_def = psi_prime_t_def.instantiate({t:one})

In [ ]:
# simplify the 2^(-0) in the exponential
psi_prime_1_def_simp = psi_prime_1_def.inner_expr().rhs.simplify()

Then show that the summation formula also gives the same qbit result

In [ ]:
sum_0_to_1 = base_case.rhs

In [ ]:
sum_0_to_1_processed_01 = sum_0_to_1.partition_first()

In [ ]:
# could eventually be handled through automation?
scalar_id_for_ket

In [ ]:
scalar_id_for_ket_inst = scalar_id_for_ket.instantiate({k: zero})

In [ ]:
sum_0_to_1_processed_02 = scalar_id_for_ket_inst.sub_right_side_into(sum_0_to_1_processed_01)

In [ ]:
# finish off the Base Case
base_case_jdgmt = sum_0_to_1_processed_02.sub_left_side_into(psi_prime_1_def_simp)

### Inductive Step

In [ ]:
inductive_step = induction_inst.antecedent.operands[1]

In [ ]:
defaults.assumptions = defaults.assumptions + inductive_step.conditions.entries

First, split the summation:
$\sum_{k=0}^{2^{t+1}-1} e^{2\pi i \varphi k} |k\rangle_{t+1} = \sum_{k=0}^{2^{t}-1} e^{2\pi i \varphi k} |k\rangle_{t+1} + \sum_{k=2^{t}}^{2^{t+1}-1} e^{2\pi i \varphi k} |k\rangle_{t+1}$

In [ ]:
# this requires knowing that 0 ≤ 2^t - 1 < 2^{t+1} - 1, proven earlier
summation_split = inductive_step.instance_expr.rhs.partition(two_pow_t_less_one)

Then shift the second summation of that split, so that the two summations then have the same index domain:

In [ ]:
# grab the second summation of the summation split
summation_split_rhs_sum = summation_split.rhs.operands[1]

In [ ]:
# recall the following simplification engineered earlier in the notebook:
index_shift_simplification

In [ ]:
rhs_sum_shifted = summation_split_rhs_sum.shifting(Neg(Exp(two, t)))

In [ ]:
rhs_sum_shifted = summation_split_rhs_sum.shifting(Neg(Exp(two, t)), replacements=[index_shift_simplification])

Substitute that shifted-and-simplified summation back into the split-summation expression from earlier, to produce the sum of two summations where the summations have matching lower bounds and matching upper bounds:

In [ ]:
summation_split_shifted = rhs_sum_shifted.sub_right_side_into(summation_split)

We want to rewrite the summand of that 2nd summation now by:

(1) expanding the exponential term; and

(2) rewriting the $|k+2^t{\rangle}_{t+1}$ ket as $|1\rangle \otimes |k{\rangle}_t$.

This takes some work, as these expressions involve the index $k$ and thus we need to eventually use `instance_substitute` on the inner_expr().

In [ ]:
# grab and label the summand to be processed:
summand_processed = rhs_sum_shifted.rhs.summand

In [ ]:
# expand the exponent in the exponential, in preparation for separating exponential into 2 factors:
summand_processed = summand_processed.inner_expr().operands[0].exponent.distribution(
        4, assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
# summand_processed = summand_processed.inner_expr().rhs.operands[0].exponent_separate(assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
# now separate the exponential into 2 factors
summand_processed = summand_processed.inner_expr().rhs.operands[0].exponent_separate(
        assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
# commute the ket label on the rhs to format it for replacement
summand_processed = summand_processed.inner_expr().rhs.operands[1].label.commute(
        assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
# Note this tensor product expansion axiom:
multi_tensor_prod_induct_1

In [ ]:
multi_tensor_prod_induct_1_inst = multi_tensor_prod_induct_1.instantiate(
        {t: t, k: k},
        assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))] )

In [ ]:
summand_processed = multi_tensor_prod_induct_1_inst.sub_left_side_into(summand_processed)

In [ ]:
# commute the exponential factors
summand_processed = summand_processed.inner_expr().rhs.operands[0].commute(
        assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
# generalize in preparation for using for instance_substitute
summand_processed_generalized = summand_processed.generalize(
        k, domain=Interval(zero, subtract(Exp(two, t), one)))

In [ ]:
summation_split_shifted = summation_split_shifted.inner_expr().rhs.operands[1].instance_substitute(
        summand_processed_generalized)

We also then want to:

(1) pull the tensor product out of the 2nd summation, and

(2) pull the non-$k$-dependent exponential factor out of the 2nd summation.

In [ ]:
# for convenience:
temp_factors = summation_split_shifted.rhs.operands[1].summand.operands[0]

In [ ]:
# for convenience:
temp_factor_01 = temp_factors.operands[0]

In [ ]:
# for convenience:
temp_factor_02 = temp_factors.operands[1]

In [ ]:
tensor_prod_linearity

In [ ]:
tensor_prod_sub = tensor_prod_linearity.instantiate(
        {j: k, a: Ket(one), b: RegisterKet(j, t), c: zero,
         d: subtract(Exp(two, t), one), t: t, fk: temp_factors},
        assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
summation_split_shifted = tensor_prod_sub.sub_right_side_into(summation_split_shifted)

In [ ]:
distributive_summation_spec

In [ ]:
distributive_summation_spec_inst = distributive_summation_spec.instantiate(
        {j:k, c:zero, d:subtract(Exp(two, t), one), x:temp_factor_01, gk:temp_factor_02, fk:RegisterKet(k, t)})

In [ ]:
summation_split_shifted = distributive_summation_spec_inst.sub_right_side_into(summation_split_shifted)

Now we want to effect a substitution into the first summation on the rhs, taking $|k{\rangle}_{t+1}$ to $|0\rangle{}\otimes|k{\rangle}_{t}$. As with the earlier effort inside the 2nd summation, this is somewhat challenging because the replacement involves an expression containing the index $k$ and thus eventually requires an `instance_substitute` step.

In [ ]:
# our axiom/theorem to apply (in reverse)
multi_tensor_prod_induct_0

In [ ]:
# our axiom/theorem instantiated
multi_tensor_prod_induct_0_inst_reversed = multi_tensor_prod_induct_0.instantiate(
        {t: t, k: k},
        assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))]).derive_reversed()

In [ ]:
first_summand = summation_split_shifted.rhs.operands[0].summand

In [ ]:
first_summand_judgment = multi_tensor_prod_induct_0_inst_reversed.substitution(first_summand.inner_expr().operands[1])

In [ ]:
first_summand_judgment_gen = first_summand_judgment.generalize(k, domain=Interval(zero, subtract(Exp(two, t), one)))

In [ ]:
summation_split_shifted = summation_split_shifted.inner_expr().rhs.operands[0].instance_substitute(first_summand_judgment_gen)

Next we want to pull the $|0\rangle$ out of the first summation, so again we use tensor_prod_linearity:

In [ ]:
tensor_prod_linearity

In [ ]:
tensor_prod_sub = tensor_prod_linearity.instantiate(
        {j: k, a: Ket(zero), b: RegisterKet(j, t), c: zero,
         d: subtract(Exp(two, t), one), t: t, fk: temp_factor_02},
        assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
summation_split_shifted = tensor_prod_sub.sub_right_side_into(summation_split_shifted)

We need a few more manipulations of that second term on the rhs: pulling the exponential factor out to the front and reassociating.

In [ ]:
factor_complex_scalar_from_tensor_prod

In [ ]:
# for convenience, name that 2nd summation on the rhs
the_summation_factor = summation_split_shifted.rhs.operands[1].operands[1].operands[1]

In [ ]:
temp_factored_tensor_prod = factor_complex_scalar_from_tensor_prod.instantiate(
        {m: one, n: zero, alpha: temp_factor_01, x: (Ket(one),), y: the_summation_factor,
        z:()},
        assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
summation_split_shifted = temp_factored_tensor_prod.sub_right_side_into(summation_split_shifted)

In [ ]:
scalar_tensor_associativity

In [ ]:
scalar_tensor_associativity_inst = scalar_tensor_associativity.instantiate(
        {alpha: temp_factor_01, x: Ket(one), y: the_summation_factor})

In [ ]:
summation_split_shifted = scalar_tensor_associativity_inst.sub_right_side_into(summation_split_shifted)

In [ ]:
distribute_tensor_prod_over_sum

In [ ]:
from proveit import Variable
i_var = Variable('i')

In [ ]:
distribute_tensor_prod_over_sum_inst = distribute_tensor_prod_over_sum.instantiate(
        {i_var: zero, j: two, k: one, y: (Ket(zero), Mult(temp_factor_01, Ket(one))), z: (the_summation_factor, )},
        assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
summation_split_shifted = distribute_tensor_prod_over_sum_inst.sub_left_side_into(summation_split_shifted)

In [ ]:
p_prime_r_def

In [ ]:
p_prime_t = p_prime_r_def.instantiate({r:t})

In [ ]:
summation_split_shifted = p_prime_t.sub_left_side_into(summation_split_shifted)

In [ ]:
# psi_prime_var_t_formula

In [ ]:
# Recall our inductive hypothesis:
for item in defaults.assumptions:
    if isinstance(item, Equals):
        inductive_hypothesis = item
inductive_hypothesis

In [ ]:
summation_split_shifted = inductive_hypothesis.sub_left_side_into(summation_split_shifted)

In [ ]:
psi_prime_expansion_inst = psi_prime_expansion.instantiate({t: t})

In [ ]:
psi_prime_t_plus_1_formula = psi_prime_expansion_inst.sub_left_side_into(summation_split_shifted).derive_reversed()

In [ ]:
# recall the inductive step:
inductive_step

In [ ]:
# we have effectively proved the inductive step:
inductive_step.prove()

In [ ]:
induction_inst

In [ ]:
# we should now have enough to prove the psi_prime_var_t_formula for all t ≥ 1
inductively_proven_formula = induction_inst.derive_consequent()

In [ ]:
# recall that t_ (i.e. t literal) represents the number of qubits in the first register;
# thus t_ is a NaturalPos:
t_in_natural_pos

In [ ]:
inductively_proven_formula.instantiate({t: t_})

In [ ]:
%qed